In [1]:
!pip install scrapling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 4.7 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 10.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.5/38.5 MB 1.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.0/873.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.2/732.2 kB 11.8 MB/s eta 0:00:00


In [3]:
from scrapling import Fetcher

input-network.zip             OK!
browser-helper-file.json      OK!
headers-order.json            OK!
header-network.zip            OK!
fingerprint-network.zip       OK!


[2025-02-23 15:27:15] INFO: Fetched (200) <GET https://news.yahoo.com/> (referer: https://www.google.com/search?q=yahoo)


In [139]:
from bs4 import BeautifulSoup
from scrapling import Fetcher
fetcher = Fetcher(auto_match=False)
page = fetcher.get('https://news.yahoo.com/', stealthy_headers=True)

html = str(page.html_content)
html = html
soup = BeautifulSoup(html, 'html.parser')

category_label = soup.find('strong', {'data-test-locator': 'stream-item-category-label'}).text.strip()

headline_tag = soup.find('h3', {'data-test-locator': 'stream-item-title'}).find('a')
news_headline = headline_tag.text.strip()
href_link = headline_tag['href']
full_link = f"https://news.yahoo.com{href_link}" if href_link.startswith('/') else href_link

summary_tag = soup.find('p', {'data-test-locator': 'stream-item-summary'})
summary = summary_tag.text.strip() if summary_tag else "No summary available"

print("Category Label:", category_label)
print("News Headline:", news_headline)
print("Href Link:", full_link)
print("Summary:", summary)

[2025-02-23 15:28:03] INFO: Fetched (200) <GET https://news.yahoo.com/> (referer: https://www.google.com/search?q=yahoo)


Category Label: US
News Headline: A South Carolina man is sentenced to life after confession on stand also led to his friend's arrest
Href Link: https://news.yahoo.com/news/south-carolina-man-sentenced-life-201603395.html
Summary: A South Carolina man who provided detailed testimony of how he brutally stabbed his friend’s ex-wife 35 times was found guilty of murder and sentenced to life in prison despite his claims the killing was justified because a child was in danger.  Zachary Hughes’ testimony this week not only led to his conviction but gave authorities enough evidence to arrest and charge his friend before Hughes even left the witness stand Wednesday.  The body of 41-year-old Christina Parcell was found in a Greenvi


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
    """
})

driver.get("https://news.yahoo.com/")
time.sleep(3)  # Wait for initial load

def scroll_down():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)  # Wait for new content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_down()

titles = []
links = []
summaries = []
categories = []

articles = driver.find_elements(By.CSS_SELECTOR, '[data-test-locator="stream-item"]')
for article in articles:
    try:
        category = article.find_element(By.CSS_SELECTOR, 'strong[data-test-locator="stream-item-category-label"]').text.strip()
        title_element = article.find_element(By.CSS_SELECTOR, 'h3[data-test-locator="stream-item-title"] a')
        title = title_element.text.strip()
        link = title_element.get_attribute("href")
        summary_element = article.find_element(By.CSS_SELECTOR, 'p[data-test-locator="stream-item-summary"]')
        summary = summary_element.text.strip() if summary_element else "No summary available"
        
        categories.append(category)
        titles.append(title)
        links.append(link)
        summaries.append(summary)
    except Exception as e:
        continue

driver.quit()

df = pd.DataFrame({
    "Category": categories,
    "Headline": titles,
    "Link": links,
    "Summary": summaries
})
df.to_csv("yahoo_news_dataset.csv", index=False)

print("Dataset saved as yahoo_news_dataset.csv")

Dataset saved as yahoo_news_dataset.csv


In [160]:
df

,Category,Headline,Link,Summary
0,Business,We've entered the era of scaredy-cat capitalism,https://news.yahoo.com/news/finance/news/weve-...,Trump's bum-rush first month in office has lef...
1,World,A Chinese wargame hints it can blind a cutting...,https://news.yahoo.com/news/chinese-wargame-hi...,"In a simulation China made public, a powerful ..."
2,World,Russia's real strongman: Meet the oligarch who...,https://news.yahoo.com/news/russias-real-stron...,"Part Steve Bannon, part Elon Musk, Konstantin ..."
3,World,Pope Francis had peaceful night's rest at hosp...,https://news.yahoo.com/news/pope-francis-had-p...,"Pope Francis, 88, rested during a peaceful nin..."
4,Business,Two founders built a jobs board for Al agents....,https://news.yahoo.com/news/tech/two-founders-...,AI agents are slowly joining the workforce — b...
5,US,Southern California man arrested for hit-and-r...,https://news.yahoo.com/news/southern-californi...,Police arrested a Southern California man who ...
6,US,Connecticut cannibal killer given conditional ...,https://news.yahoo.com/news/connecticut-cannib...,An institutionalized man who confessed to the ...
7,US,Driver arrested after crashing into pole durin...,https://news.yahoo.com/news/driver-arrested-cr...,A suspect driver was arrested after crashing d...
8,Celebrity,Savannah Chrisley says she 'lost some deals' o...,https://news.yahoo.com/news/entertainment/sava...,Savannah Chrisley shared she lost lucrative bu...
9,US,Here's how federal workers are responding to t...,https://news.yahoo.com/news/heres-federal-work...,Federal workers across agencies told Business ...


In [152]:
!pip install undetected_chromedriver fake_useragent pyautogui

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.54-py3-none-any.whl size=37637 sha256=36e3236e90e1c153dc6d8f4699d33565519e3697a2f065f392e57892b547444c
  Stored in directory: /Users/ayushsingh/Library/Caches/pip/wheels/23/a7/1c/5a51aaff3bbe110be4ddf766d429cc9d2fae7a72fc1b843e56
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl size=11064 sha256=b2eb3f60884

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys
# from webdriver_manager.chrome import ChromeDriverManager
# import pandas as pd
# import time
# import random
# import undetected_chromedriver as uc
# from fake_useragent import UserAgent

# # Configure Chrome options to evade detection
# ua = UserAgent()
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode
# chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument(f"user-agent={ua.random}")

# # Initialize WebDriver
# service = Service(ChromeDriverManager().install())
# driver = uc.Chrome(service=service, options=chrome_options)

# # Bypass detection
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
#     "source": """
#     Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
#     """
# })

# driver.execute_cdp_cmd("Network.setUserAgentOverride", {"userAgent": ua.random})
# driver.execute_cdp_cmd("Network.enable", {})
# driver.execute_cdp_cmd("Network.setExtraHTTPHeaders", {"headers": {"WebRTC": "disabled"}})

# driver.execute_cdp_cmd("Emulation.setGeolocationOverride", {
#     "latitude": random.uniform(-90, 90),
#     "longitude": random.uniform(-180, 180),
#     "accuracy": 100
# })

# # Open the target URL
# driver.get("https://news.yahoo.com/")
# time.sleep(random.uniform(2, 5))  # Randomized delay for initial load

# # Scroll to load more content
# def scroll_down():
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         # Scroll to the bottom of the page
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         time.sleep(random.uniform(2, 5))  # Randomized delay
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height

# scroll_down()

# # Extract news articles
# titles = []
# links = []
# summaries = []
# categories = []

# articles = driver.find_elements(By.CSS_SELECTOR, '[data-test-locator="stream-item"]')
# for article in articles:
#     try:
#         category = article.find_element(By.CSS_SELECTOR, 'strong[data-test-locator="stream-item-category-label"]').text.strip()
#         title_element = article.find_element(By.CSS_SELECTOR, 'h3[data-test-locator="stream-item-title"] a')
#         title = title_element.text.strip()
#         link = title_element.get_attribute("href")
#         summary_element = article.find_element(By.CSS_SELECTOR, 'p[data-test-locator="stream-item-summary"]')
#         summary = summary_element.text.strip() if summary_element else "No summary available"
        
#         categories.append(category)
#         titles.append(title)
#         links.append(link)
#         summaries.append(summary)
#     except Exception as e:
#         continue

# # Close WebDriver
# driver.quit()

# # Save to CSV
# df = pd.DataFrame({
#     "Category": categories,
#     "Headline": titles,
#     "Link": links,
#     "Summary": summaries
# })
# df.to_csv("yahoo_news_dataset.csv", index=False)

# print("Dataset saved as yahoo_news_dataset.csv")

Dataset saved as yahoo_news_dataset.csv


In [142]:
!pip install webdriver_manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
